In [1]:
from collections import defaultdict
from datetime import datetime
import dill
from itertools import permutations, combinations
import json
from operator import itemgetter
import os
import pickle
import random
import re
import time

import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text
sns.set(style='ticks', font_scale=1.2)

import little_mallet_wrapper as lmw

In [2]:
data_directory_path   = '/Volumes/Passport-1/data/birth-control'
output_directory_path = '/Volumes/Passport-1/output/birth-control'

<br><br>

# Sample Twitter posts for labeling in Prodigy

In [3]:
twitter_posts_df = pd.read_csv(data_directory_path + '/twitter/new-from-api/posts/twitter_posts.csv')

In [4]:
len(twitter_posts_df)

528923

In [5]:
twitter_posts_df.sample(3)

,Unnamed: 0,source,text,token_text,text_type,date,year,id,conversation_id,retweet_count,reply_count,like_count,quote_count
461125,553343,twitter post,http://t.co/wwZRYN0D FDA approves Watson Vestu...,http //t co/wwzrynNUMd fda approves watson ves...,pill,2011-11-30T14:34:27.000Z,2011,141887822767468544,141887822767468544,0,0,0,0
371455,431311,twitter post,Contraceptive pill for men http://t.co/h6mGHt4JAe,contraceptive pill men http //t co/hNUMmghtNUMjae,pill,2015-01-18T05:08:23.000Z,2015,556679471982714880,556679471982714880,0,0,0,0
507647,611796,twitter post,Open Question: Im really scared to take the bi...,open question really scared take birth control...,pill,2010-01-30T04:42:58.000Z,2010,8399563420,8399563420,0,0,0,0


In [6]:
sampled_posts_df = twitter_posts_df.sample(1000)

In [7]:
len(sampled_posts_df.index)

1000

In [8]:
sampled_posts_df.sample(3)

,Unnamed: 0,source,text,token_text,text_type,date,year,id,conversation_id,retweet_count,reply_count,like_count,quote_count
74531,78832,twitter post,"New, more affordable IUDs by nonprofit @Medici...",new affordable iuds nonprofit medicinesNUM usa...,iud,2017-08-26T18:27:06.000Z,2017,901511336135446528,901511336135446528,0,0,1,0
54207,56619,twitter post,#IUD 'easily the most effective' #birthcontrol...,iud easily effective birthcontrol teen girls c...,iud,2018-06-01T06:20:00.000Z,2018,1002434569499488256,1002434569499488256,0,0,0,0
270304,303738,twitter post,Contraceptive implant: Does anyone use this?. ...,contraceptive implant anyone use would like av...,implant,2011-08-16T20:19:59.000Z,2011,103561662220611584,103561662220611584,0,0,0,0


In [9]:
sampled_posts_df.to_csv('/Volumes/Passport-1/data/birth-control/labeling/sampled-twitter-posts.csv')

<br><br>

# Sample Twitter replies for Prodigy

In [10]:
twitter_replies_df = pd.read_csv(data_directory_path + '/twitter/new-from-api/replies/twitter_replies.csv')

In [11]:
len(twitter_replies_df.index)

271047

In [12]:
twitter_replies_df.sample(3)

,Unnamed: 0,source,text,token_text,text_type,date,year,id,conversation_id,retweet_count,reply_count,like_count,quote_count
77002,77786,twitter post,@easterly_dalton I’m not saying it’s easy. Sur...,easterly dalton saying easy sure nervous iud d...,iud,2018-07-09T02:52:52.000Z,2018,1016153182844719104,1016151622152581120,0,0,0,0
93460,94366,twitter post,@Spacebunnyday With that reasoning.. you shoul...,spacebunnyday reasoning pill condoms iud femal...,iud,2017-10-29T16:33:58.000Z,2017,924675690762346496,924673611541155841,0,2,0,1
63596,64058,twitter post,@BerniceBee Kalani is making SOOO many excuses...,bernicebee kalani making sooo many excuses lik...,iud,2018-12-17T01:33:02.000Z,2018,1074477537777078272,1074477130308837382,0,0,0,0


In [13]:
sampled_replies_df = twitter_replies_df.sample(1000)

In [14]:
len(sampled_replies_df.index)

1000

In [15]:
sampled_replies_df.sample(3)

,Unnamed: 0,source,text,token_text,text_type,date,year,id,conversation_id,retweet_count,reply_count,like_count,quote_count
34224,34398,twitter post,@DAY____KING Same wozzap!! Praying my IUD stay...,day king wozzap praying iud stays put,iud,2019-07-10T09:45:51.000Z,2019,1148891070064930817,1148448354424229889,0,0,0,0
157415,159508,twitter post,@DrOlgaTheOBGYN @markhoffmanmd @JamesCaseyMD @...,drolgatheobgyn markhoffmanmd jamescaseymd mals...,iud,2020-12-16T16:28:31.000Z,2020,1339246036813029377,1339179649805508608,0,1,4,0
261083,273068,twitter post,Long-term contraceptive pill use doubles glauc...,long term contraceptive pill use doubles glauc...,pill,2013-11-19T09:24:52.000Z,2013,402729176991817728,402725259826458624,0,0,0,0


In [16]:
sampled_replies_df.to_csv('/Volumes/Passport-1/data/birth-control/labeling/sampled-twitter-replies.csv')